Test a bunch of notebooks. Bioinfo notebooks are at ~/repos/bioinfo/test.ipynb

In [1]:
from k1lib.imports import *
from k1lib.bioinfo.cli import *
from IPython.display import clear_output

In [2]:
def execute(nb:str):
    print(nb)
    k1lib.executeNb(nb, globals())
    clear_output()

In [3]:
base = "/home/kelvin/repos/labs/k1lib/k1lib"
tests = []
for i in range(5): tests += glob.glob(f"{base}{'/*' * i}.ipynb")
tests.remove(f"{base}/test.ipynb")
for i, test in enumerate(tests):
    print(i)
    execute(test)

In [4]:
tuts = glob.glob("/home/kelvin/repos/labs/k1lib/docs/tutorials/*.ipynb")
execute(tuts[2]); execute(tuts[1])
execute(tuts[0]) # heaviest

In [5]:
execute("/home/kelvin/repos/labs/projects/char rnn/1) init/1) init.ipynb")

In [6]:
execute("/home/kelvin/repos/labs/projects/osic/osic.ipynb")

In [7]:
execute("/home/kelvin/repos/labs/projects/osic/imgs.ipynb")

In [8]:
k1lib.beep()